In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Data file paths
mushroom_path = "../secondary_data_no_miss.csv"

# Read the data files
mushroom_data = pd.read_csv(mushroom_path, sep = ";")

mushroom_data

In [ ]:
# replace all letters with actual value

mushroom_data = mushroom_data.replace(
    {
    'class': {'p': 'poisonous', 'e': 'edible'},
    'cap-shape': {'b': 'bell', 'c': 'conical', 'x': 'convex', 'f': 'flat', 's': 'sunken', 'p': 'spherical', 'o': 'others'},
    'cap-surface': {'i': 'fibrous', 'g': 'grooves', 'y': 'scaly', 's': 'smooth', 'd': 'dry', 'h': 'shiny', 'l': 'leathery', 'k': 'silky', 't': 'sticky', 'w': 'wrinkled', 'e': 'fleshy'},
    'cap-color': {'n': 'brown', 'b': 'buff', 'g': 'gray', 'r': 'green', 'p': 'pink', 'u': 'purple', 'e': 'red', 'w': 'white', 'y': 'yellow', 'l': 'blue', 'o': 'orange', 'k': 'black'},
    'gill-attachment': {'a': 'adnate', 'x': 'adnexed', 'd': 'decurrent', 'e': 'free', 's': 'sinuate', 'p': 'pores', 'f': 'none', '?': 'unknown'},
    'gill-spacing': {'c': 'close', 'd': 'distant', 'f': 'none'},
    'gill-color': {'n': 'brown', 'b': 'buff', 'g': 'gray', 'r': 'green', 'p': 'pink', 'u': 'purple', 'e': 'red', 'w': 'white', 'y': 'yellow', 'l': 'blue', 'o': 'orange', 'k': 'black', 'f': 'none'},
    'stem-color': {'n': 'brown', 'b': 'buff', 'g': 'gray', 'r': 'green', 'p': 'pink', 'u': 'purple', 'e': 'red', 'w': 'white', 'y': 'yellow', 'l': 'blue', 'o': 'orange', 'k': 'black', 'f': 'none'},
    'ring-type': {'c': 'cobwebby', 'e': 'evanescent', 'r': 'flaring', 'g': 'grooved', 'l': 'large', 'p': 'pendant', 's': 'sheathing', 'z': 'zone', 'y': 'scaly', 'm': 'movable', 'f': 'none', '?': 'unknown'},
    'habitat': {'g': 'grasses', 'l': 'leaves', 'm': 'meadows', 'p': 'paths', 'h': 'heaths', 'u': 'urban', 'w': 'waste', 'd': 'woods'},
    'season': {'s': 'spring', 'u': 'summer', 'a': 'autumn', 'w': 'winter'}
    })

# convert t/f columns to boolean.
mushroom_data['does-bruise-or-bleed'] = mushroom_data['does-bruise-or-bleed']=='t'
mushroom_data['has-ring'] = mushroom_data['has-ring']=='t'
mushroom_data

## Run KNN on the three original quantitative data columns with standard and MinMax scalers

In [ ]:
# Establish X and y

y = mushroom_data["class"]
target_names = ["edible", "poisonous"]

# Need to drop all non-integer columns for KNN
# X = mushroom_data.drop("class", axis=1)
X = mushroom_data[["cap-diameter", "stem-height", "stem-width"]]
X.head()

In [ ]:
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

from sklearn.preprocessing import StandardScaler
# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)

# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# from pickle import dump, load
# dump(X_scaler, open('knn_scaler.pkl', 'wb'))

# joblib.dump(X_scaler, 'mushroom_scaler_knn.pkl')

In [ ]:
# Loop through different k values to see which has the highest accuracy
# NB: Standard scaled, 3 quantitative columns
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 30, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 30, 2), train_scores, marker='o')
plt.plot(range(1, 30, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 13 seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train_scaled, y_train)
print('For a standard scaled KNN model on the three original quantitative columns: k=17 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

In [ ]:
# Save model

import joblib
# import pickle
# from pickle import dump, load
# pickle.dump(knn, open("knn_standard_three.pkl", "wb"))

# # Save the model as a pickle in a file
# joblib.dump(knn, 'knn_standard_three.pkl')

# # Load the model from the file
# knn_from_joblib = joblib.load('knn_standard_three.pkl')

# # Use the loaded model to make predictions
# knn_from_joblib.predict(X_test)



In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Create a StandardScater model and fit it to the training data

X_scaler = MinMaxScaler().fit(X_train)

# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Loop through different k values to see which has the highest accuracy
# NB: Standard scaled, 3 quantitative columns
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 30, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 30, 2), train_scores, marker='o')
plt.plot(range(1, 30, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 13 seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=17)
knn.fit(X_train_scaled, y_train)
print('For a min/max scaled KNN model on the three original quantitative columns: k=17 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

In [ ]:
# # Save the model as a pickle in a file
# joblib.dump(knn, 'knn_minmax_three.pkl')

## Get dummy columns for categorical data and run KNN (standard, MinMax) on all data together

In [ ]:

mushroom_category = pd.get_dummies(data=mushroom_data, columns=[
                                                'cap-surface',
                                                'cap-color',
                                                'does-bruise-or-bleed',
                                                'gill-attachment',
                                                'gill-spacing',
                                                'gill-color',
                                                'cap-shape',
                                                'stem-color',
                                                'has-ring',
                                                'ring-type',
                                                'habitat',
                                                'season'])

# mushroom_category.to_csv("mushroom_category.csv", index=False)

mushroom_category


In [ ]:
y = mushroom_category["class"]
target_names = ["edible", "poisonous"]

# X = mushroom_data.drop("class", axis=1)
X = mushroom_category.drop("class", axis=1)
X.head()

In [ ]:
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# StandardScaler with get_dummies columns

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 10, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 10, 2), train_scores, marker='o')
plt.plot(range(1, 10, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 3 seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_scaled, y_train)
print('For a standard scaled KNN model on all columns: k=3 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

In [ ]:
# # Save the model as a pickle in a file
# joblib.dump(knn, 'knn_standard_all.pkl')

# MinMaxScaler with get_dummies columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Create a StandardScaler model and fit it to the training data

X_scaler = MinMaxScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 10, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 10, 2), train_scores, marker='o')
plt.plot(range(1, 10, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 2 seems to be the best choice for this dataset
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train_scaled, y_train)
print('For a MinMax scaled KNN model on all columns: k=2 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))

## Initial stab at neural network with MinMaxScaler

In [ ]:
# Let's try a neural network

from tensorflow.keras.utils import to_categorical
# import numpy as np

In [ ]:
X = mushroom_category.drop("class", axis=1)
y = mushroom_category["class"]
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from tensorflow.keras.utils import to_categorical

In [ ]:
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
import joblib

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# joblib.dump(X_scaler, 'mushroom_scaler.pkl')

# from pickle import dump
# dump(X_scaler, open('mush_nn_scale.pkl', 'wb'))

In [ ]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [ ]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [ ]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Make a sequential model
model = Sequential()
# Add the first hidden layer with 6 nodes
model.add(Dense(units=6, activation='relu', input_dim=92))
# Add the output layer with 2 outputs
model.add(Dense(units=2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
# # Save the model
# model.save("mushroom_trained.h5")

# import tensorflow as tf
# mushroom_model = tf.keras.models.load_model('mushroom_trained.h5')

# mushroom_model.predict(new_data)

# import pickle

# mushroom_model = 'mushroom_model.sav'
# pickle.dump(model, open(mushroom_model, 'wb'))

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
mushroom_model = load_model("mushroom_trained.h5")

In [ ]:
model_loss, model_accuracy = mushroom_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
model_loss.dtype

In [ ]:
# Grab just one data point to test with
test = np.expand_dims(X_test_scaled[1], axis=0)
test.shape

In [ ]:
# Make a prediction
print(f"Predicted class: {model.predict_classes(test)}")

In [ ]:
X_test_scaled[1]